# XoT Tutorial

Import OpenAI model with langchain.

In [1]:
import os
from dotenv import load_dotenv

import langchain
from langchain_openai import AzureChatOpenAI

from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

langchain.debug = True # set verbose mode to True

load_dotenv()

langchain_llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("HKUST_OPENAI_BASE"),
    openai_api_key=os.getenv("HKUST_OPENAI_KEY"),
    api_version=os.getenv("HKUST_OPENAI_API_VERSION"),
    openai_api_type="azure",
    deployment_name=os.getenv("HKUST_OPENAI_DEPLOYMENT_NAME"),
    verbose=True
)

## Direct Use of LLM

* Practice the use of LLM with langchain.
* Intent classifier: does the user want to have its tablet repaired?

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

intent_classifier_template = """
You are a customer service chatbot help to decide if the user want to repair its device. Base on the user's description, reply "Yes" if the user want to repair the device, "No" if the user do not want to repair the device, and "End" if the user want to end the conversation.

{format_instructions}

The user's input is: {user_input}
"""


class Intent(BaseModel):
    want_to_repair: str = Field(
        description="whether the user want to repair the device", example="Yes"
    )


intent_classifier_parser = JsonOutputParser(pydantic_object=Intent)

intent_classifier_prompt = PromptTemplate(
    template=intent_classifier_template,
    input_variables=["user_input"],
    partial_variables={
        "format_instructions": intent_classifier_parser.get_format_instructions()
    },
)

intent_classifier_chain = (
    intent_classifier_prompt | langchain_llm | intent_classifier_parser
)

In [3]:
intent_classifier_chain.invoke(
    {"user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."}
)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "user_input": "My ipad cannot connect to bluetooth, I'm here to get some support."
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are a customer service chatbot help to decide if the user want to repair its device. Base on the user's description, reply \"Yes\" if the user want to repair the device, \"No\" if the user do not want to repair the device, and \"End\" if the user want to end the conversation.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\"

{'want_to_repair': 'Yes'}

## Chain of Thoughts Prompting

* Practice the use of CoT with langchain.
* Use warranty checker as an example.

In [4]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

warranty_checker_examples = [
    {
        "ai_question": "When did you purchase this item?",
        "human_response": "A month ago",
        "ai_response": "Analysis: Let's think step by step. All products come with a 90-day warranty since purchase. A month is within the 90-day time span. The product must have warranty. Warranty: Yes. Utterance: Great! Your product has warranty.",
    },
    {
        "ai_question": "When did you purchase this item?",
        "human_response": "Last year.",
        "ai_response": "Analysis: Let's think step by step. All products come with a 90-day warranty. A year is more than this time span. However, we are not sure if the user purchased additional warranty. We are not sure if the product has warranty. Warranty: Unsure. Utterance: Did you purchase additional warranty?",
    },
]

warranty_checker_example_template = ChatPromptTemplate.from_messages(
    [
        ("ai", "{ai_question}"),
        ("human", "{human_response}"),
        ("ai", "{ai_response}"),
    ]
)

warranty_checker_fewshot_examples = FewShotChatMessagePromptTemplate(
    example_prompt=warranty_checker_example_template,
    examples=warranty_checker_examples,
)


warranty_checker_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful agent that help to decide if a product is out of warranty. Note that all product come with a 90-day warranty since purchase. Customer can also purchase additional warranty that extends it to 2 years. Today is May 20th, 2024."
        ),
        warranty_checker_fewshot_examples,
       ("human", "{user_input}")
    ]
)

warranty_checker_chain = warranty_checker_prompt | langchain_llm

In [5]:
warranty_checker_fewshot_examples.format()

"AI: When did you purchase this item?\nHuman: A month ago\nAI: Analysis: Let's think step by step. All products come with a 90-day warranty since purchase. A month is within the 90-day time span. The product must have warranty. Warranty: Yes. Utterance: Great! Your product has warranty.\nAI: When did you purchase this item?\nHuman: Last year.\nAI: Analysis: Let's think step by step. All products come with a 90-day warranty. A year is more than this time span. However, we are not sure if the user purchased additional warranty. We are not sure if the product has warranty. Warranty: Unsure. Utterance: Did you purchase additional warranty?"

In [6]:
warranty_checker_chain.invoke({"user_input": "I bought this item on May 1st, 2024."})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "user_input": "I bought this item on May 1st, 2024."
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "user_input": "I bought this item on May 1st, 2024."
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful agent that help to decide if a product is out of warranty. Note that all product come with a 90-day warranty since purchase. Customer can also purchase additional warranty that extends it to 2 years. Today is May 20th, 2024.\nAI: When did you purchase this item?\nHuman: A month ago\nAI: Analysis: Let's think step by step. All products come with a 90-day warranty since purchase. A month is within the 90-day time span. The product must have warranty. Warranty: Yes.

AIMessage(content="Analysis: Let's think step by step. All products come with a 90-day warranty since purchase. Today is May 20th, 2024, which means it has been less than 90 days since the purchase. The product is still within the warranty period. Warranty: Yes. Utterance: Great! Your product is still under warranty.", response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 243, 'total_tokens': 315}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence'